In [ ]:
# === V4: OPTIMIZED BASELINE + 2000 TERMS ===
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import numpy as np
import pandas as pd
import gc

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

CONFIG = {
    "n_terms": 2000,  # INCREASED from 1500
    "paths": {
        "train_emb": "/kaggle/input/esm-dataset/train_embeds.npy",
        "train_ids": "/kaggle/input/esm-dataset/train_ids.npy",
        "test_emb": "/kaggle/input/esm-dataset/test_embeds.npy",
        "test_ids": "/kaggle/input/esm-dataset/test_ids.npy",
        "train_terms": "/kaggle/input/cafa-6-protein-function-prediction/Train/train_terms.tsv",
        "go_obo": "/kaggle/input/cafa-6-protein-function-prediction/Train/go-basic.obo"
    }
}

# LOAD DATA
print("Loading data...")
train_emb = np.load(CONFIG["paths"]["train_emb"]).astype(np.float32)
train_ids = np.load(CONFIG["paths"]["train_ids"])
test_emb = np.load(CONFIG["paths"]["test_emb"]).astype(np.float32)
test_ids = np.load(CONFIG["paths"]["test_ids"])

print(f"Train: {train_emb.shape}, Test: {test_emb.shape}")

# Normalize
mean = train_emb.mean(axis=0)
std = train_emb.std(axis=0) + 1e-6
train_emb = (train_emb - mean) / std
test_emb = (test_emb - mean) / std

# Load terms
terms_df = pd.read_csv(CONFIG["paths"]["train_terms"], sep="\t", header=None, names=["id", "term", "aspect"])
id_to_idx = {pid: i for i, pid in enumerate(train_ids)}

# MODEL (Same as your 0.195 winner)
class SimpleModel(nn.Module):
    def __init__(self, n_feat, n_class):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_feat, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, n_class)
        )
    def forward(self, x): return self.net(x)

class SimpleData(Dataset):
    def __init__(self, X, y=None):
        self.X = torch.from_numpy(X)
        self.y = torch.from_numpy(y) if y is not None else None
    def __len__(self): return len(self.X)
    def __getitem__(self, i): return (self.X[i], self.y[i]) if self.y is not None else self.X[i]

# TRAIN
def train_aspect(aspect_char, aspect_name):
    print(f"\n>>> Training {aspect_name} ({aspect_char})...")
    
    aspect_terms = terms_df[terms_df['aspect'] == aspect_char]
    top_terms = aspect_terms['term'].value_counts().index[:CONFIG["n_terms"]].tolist()
    term_map = {t: i for i, t in enumerate(top_terms)}
    num_classes = len(top_terms)
    print(f"Using {num_classes} GO terms")
    
    label_matrix = np.zeros((len(train_ids), num_classes), dtype=np.float32)
    relevant = aspect_terms[aspect_terms['term'].isin(top_terms)]
    
    for _, row in tqdm(relevant.iterrows(), total=len(relevant), desc="Labels"):
        if row['id'] in id_to_idx:
            label_matrix[id_to_idx[row['id']], term_map[row['term']]] = 1.0
    
    ds = SimpleData(train_emb, label_matrix)
    loader = DataLoader(ds, batch_size=256, shuffle=True, num_workers=2)
    
    model = SimpleModel(1280, num_classes).to(device)
    opt = torch.optim.Adam(model.parameters(), lr=1e-3)
    loss_fn = nn.BCEWithLogitsLoss()
    
    for epoch in range(10):
        model.train()
        total = 0
        for x, y in tqdm(loader, desc=f"Epoch {epoch+1}", leave=False):
            x, y = x.to(device), y.to(device)
            opt.zero_grad()
            loss = loss_fn(model(x), y)
            loss.backward()
            opt.step()
            total += loss.item()
        print(f"Epoch {epoch+1} Loss: {total/len(loader):.4f}")
    
    torch.save(model.state_dict(), f"model_v4_{aspect_char}.pth")
    return top_terms

# Train all 3
print("\n=== TRAINING ===")
results = {}
for char, name in [('F', 'Function'), ('P', 'Process'), ('C', 'Component')]:
    results[char] = train_aspect(char, name)

print("\n Training Complete!")

# PREDICT
print("\n=== PREDICTING ===")
with open("submission_v4_2000terms.tsv", "w") as f:
    for char in ['F', 'P', 'C']:
        print(f"Predicting {char}...")
        terms = results[char]
        num_classes = len(terms)
        
        model = SimpleModel(1280, num_classes).to(device)
        model.load_state_dict(torch.load(f"model_v4_{char}.pth"))
        model.eval()
        
        loader = DataLoader(torch.from_numpy(test_emb), batch_size=1024)
        preds = []
        with torch.no_grad():
            for x in tqdm(loader):
                preds.append(torch.sigmoid(model(x.to(device))).cpu().numpy())
        
        all_preds = np.vstack(preds)
        
        for i, pid in enumerate(tqdm(test_ids)):
            top_idx = np.argpartition(all_preds[i], -70)[-70:]
            for idx in top_idx:
                if all_preds[i, idx] > 0.01:
                    f.write(f"{pid}\t{terms[idx]}\t{all_preds[i, idx]:.3f}\n")
        
        del model, preds
        gc.collect()

print("\n Predictions Complete!")


In [ ]:
# === GRAPH PROPAGATION ===
!pip install obonet networkx -q

import obonet
from tqdm import tqdm

print("Loading GO Graph...")
graph = obonet.read_obo(CONFIG["paths"]["go_obo"])
parent_map = {n: list(graph.successors(n)) for n in graph.nodes()}

print("Loading predictions...")
sub = {}
with open("submission_v4_2000terms.tsv") as f:
    for line in tqdm(f):
        p, t, s = line.strip().split("\t")
        if p not in sub: sub[p] = {}
        sub[p][t] = float(s)

print("Propagating...")
with open("submission.tsv", "w") as f:
    for pid, preds in tqdm(sub.items()):
        final = preds.copy()
        q = list(preds.keys())
        visited = set(q)
        
        while q:
            term = q.pop(0)
            for par in parent_map.get(term, []):
                if final.get(par, 0) < final[term]:
                    final[par] = final[term]
                    if par not in visited:
                        q.append(par)
                        visited.add(par)
        
        for t, s in sorted(final.items(), key=lambda x: -x[1])[:70]:
            if s > 0.001:
                f.write(f"{pid}\t{t}\t{s:.3f}\n")

print("\n✅ submission.tsv ready! Download and submit.")


In [ ]:
# === ENSEMBLE: 3 MODELS WITH DIFFERENT SEEDS ===
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import numpy as np
import pandas as pd
import gc

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

CONFIG = {
    "n_terms": 1500,  # KEEP AT 1500
    "n_models": 3,     # Train 3 different models
    "paths": {
        "train_emb": "/kaggle/input/esm-dataset/train_embeds.npy",
        "train_ids": "/kaggle/input/esm-dataset/train_ids.npy",
        "test_emb": "/kaggle/input/esm-dataset/test_embeds.npy",
        "test_ids": "/kaggle/input/esm-dataset/test_ids.npy",
        "train_terms": "/kaggle/input/cafa-6-protein-function-prediction/Train/train_terms.tsv",
        "go_obo": "/kaggle/input/cafa-6-protein-function-prediction/Train/go-basic.obo"
    }
}

# LOAD DATA
print("Loading...")
train_emb = np.load(CONFIG["paths"]["train_emb"]).astype(np.float32)
train_ids = np.load(CONFIG["paths"]["train_ids"])
test_emb = np.load(CONFIG["paths"]["test_emb"]).astype(np.float32)
test_ids = np.load(CONFIG["paths"]["test_ids"])

mean = train_emb.mean(axis=0)
std = train_emb.std(axis=0) + 1e-6
train_emb = (train_emb - mean) / std
test_emb = (test_emb - mean) / std

terms_df = pd.read_csv(CONFIG["paths"]["train_terms"], sep="\t", header=None, names=["id", "term", "aspect"])
id_to_idx = {pid: i for i, pid in enumerate(train_ids)}

# MODEL
class SimpleModel(nn.Module):
    def __init__(self, n_feat, n_class):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_feat, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, n_class)
        )
    def forward(self, x): return self.net(x)

class SimpleData(Dataset):
    def __init__(self, X, y=None):
        self.X = torch.from_numpy(X)
        self.y = torch.from_numpy(y) if y is not None else None
    def __len__(self): return len(self.X)
    def __getitem__(self, i): return (self.X[i], self.y[i]) if self.y is not None else self.X[i]

# TRAIN MULTIPLE MODELS
def train_ensemble(aspect_char, aspect_name):
    print(f"\n>>> Training {aspect_name} Ensemble...")
    
    aspect_terms = terms_df[terms_df['aspect'] == aspect_char]
    top_terms = aspect_terms['term'].value_counts().index[:CONFIG["n_terms"]].tolist()
    term_map = {t: i for i, t in enumerate(top_terms)}
    
    label_matrix = np.zeros((len(train_ids), len(top_terms)), dtype=np.float32)
    relevant = aspect_terms[aspect_terms['term'].isin(top_terms)]
    for _, row in tqdm(relevant.iterrows(), total=len(relevant)):
        if row['id'] in id_to_idx:
            label_matrix[id_to_idx[row['id']], term_map[row['term']]] = 1.0
    
    # Train N models with different seeds
    for model_idx in range(CONFIG["n_models"]):
        print(f"\n  Model {model_idx+1}/{CONFIG['n_models']}...")
        
        # Set seed for reproducibility
        torch.manual_seed(42 + model_idx)
        np.random.seed(42 + model_idx)
        
        ds = SimpleData(train_emb, label_matrix)
        loader = DataLoader(ds, batch_size=256, shuffle=True)
        
        model = SimpleModel(1280, len(top_terms)).to(device)
        opt = torch.optim.Adam(model.parameters(), lr=1e-3)
        loss_fn = nn.BCEWithLogitsLoss()
        
        for epoch in range(10):
            model.train()
            total = 0
            for x, y in loader:
                x, y = x.to(device), y.to(device)
                opt.zero_grad()
                loss = loss_fn(model(x), y)
                loss.backward()
                opt.step()
                total += loss.item()
            if epoch % 3 == 0:
                print(f"    Epoch {epoch+1}: {total/len(loader):.4f}")
        
        torch.save(model.state_dict(), f"model_ens_{aspect_char}_{model_idx}.pth")
    
    return top_terms

# Train ensembles
results = {}
for char, name in [('F', 'Function'), ('P', 'Process'), ('C', 'Component')]:
    results[char] = train_ensemble(char, name)

# PREDICT (Average all models)
print("\n=== ENSEMBLE PREDICTION ===")
with open("submission_ensemble.tsv", "w") as f:
    for char in ['F', 'P', 'C']:
        print(f"\nEnsembling {char}...")
        terms = results[char]
        
        # Collect predictions from all models
        all_model_preds = []
        for model_idx in range(CONFIG["n_models"]):
            model = SimpleModel(1280, len(terms)).to(device)
            model.load_state_dict(torch.load(f"model_ens_{char}_{model_idx}.pth"))
            model.eval()
            
            loader = DataLoader(torch.from_numpy(test_emb), batch_size=1024)
            preds = []
            with torch.no_grad():
                for x in loader:
                    preds.append(torch.sigmoid(model(x.to(device))).cpu().numpy())
            
            all_model_preds.append(np.vstack(preds))
            del model
        
        # AVERAGE predictions
        final_preds = np.mean(all_model_preds, axis=0)
        
        for i, pid in enumerate(tqdm(test_ids)):
            top_idx = np.argpartition(final_preds[i], -70)[-70:]
            for idx in top_idx:
                if final_preds[i, idx] > 0.01:
                    f.write(f"{pid}\t{terms[idx]}\t{final_preds[i, idx]:.3f}\n")
        
        del all_model_preds
        gc.collect()

print("\n✅ Ensemble complete!")


In [ ]:
!pip install obonet -q
import obonet
graph = obonet.read_obo(CONFIG["paths"]["go_obo"])
parent_map = {n: list(graph.successors(n)) for n in graph.nodes()}

sub = {}
with open("submission_ensemble.tsv") as f:
    for line in f:
        p, t, s = line.strip().split("\t")
        if p not in sub: sub[p] = {}
        sub[p][t] = float(s)

with open("submissionfinal.tsv", "w") as f:
    for pid, preds in tqdm(sub.items()):
        final = preds.copy()
        q = list(preds.keys())
        while q:
            term = q.pop(0)
            for par in parent_map.get(term, []):
                if final.get(par, 0) < final[term]:
                    final[par] = final[term]
                    if par not in preds: q.append(par)
        for t, s in sorted(final.items(), key=lambda x: -x[1])[:70]:
            if s > 0.001:
                f.write(f"{pid}\t{t}\t{s:.3f}\n")


In [ ]:
# === DIVERSE ARCHITECTURE ENSEMBLE ===
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import numpy as np
import pandas as pd
import gc

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

CONFIG = {
    "n_terms": 1500,
    "paths": {
        "train_emb": "/kaggle/input/esm-dataset/train_embeds.npy",
        "train_ids": "/kaggle/input/esm-dataset/train_ids.npy",
        "test_emb": "/kaggle/input/esm-dataset/test_embeds.npy",
        "test_ids": "/kaggle/input/esm-dataset/test_ids.npy",
        "train_terms": "/kaggle/input/cafa-6-protein-function-prediction/Train/train_terms.tsv",
        "go_obo": "/kaggle/input/cafa-6-protein-function-prediction/Train/go-basic.obo"
    }
}

# LOAD DATA
print("Loading...")
train_emb = np.load(CONFIG["paths"]["train_emb"]).astype(np.float32)
train_ids = np.load(CONFIG["paths"]["train_ids"])
test_emb = np.load(CONFIG["paths"]["test_emb"]).astype(np.float32)
test_ids = np.load(CONFIG["paths"]["test_ids"])

mean = train_emb.mean(axis=0)
std = train_emb.std(axis=0) + 1e-6
train_emb = (train_emb - mean) / std
test_emb = (test_emb - mean) / std

terms_df = pd.read_csv(CONFIG["paths"]["train_terms"], sep="\t", header=None, names=["id", "term", "aspect"])
id_to_idx = {pid: i for i, pid in enumerate(train_ids)}

# THREE DIFFERENT ARCHITECTURES
class Model_A(nn.Module):  # Original (2-layer, 512)
    def __init__(self, n_feat, n_class):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_feat, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, n_class)
        )
    def forward(self, x): return self.net(x)

class Model_B(nn.Module):  # Deeper (3-layer)
    def __init__(self, n_feat, n_class):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_feat, 1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, n_class)
        )
    def forward(self, x): return self.net(x)

class Model_C(nn.Module):  # Wider + BatchNorm
    def __init__(self, n_feat, n_class):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_feat, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(1024, n_class)
        )
    def forward(self, x): return self.net(x)

class SimpleData(Dataset):
    def __init__(self, X, y=None):
        self.X = torch.from_numpy(X)
        self.y = torch.from_numpy(y) if y is not None else None
    def __len__(self): return len(self.X)
    def __getitem__(self, i): return (self.X[i], self.y[i]) if self.y is not None else self.X[i]

# TRAIN
def train_diverse_ensemble(aspect_char, aspect_name):
    print(f"\n>>> Training {aspect_name} Diverse Ensemble...")
    
    aspect_terms = terms_df[terms_df['aspect'] == aspect_char]
    top_terms = aspect_terms['term'].value_counts().index[:CONFIG["n_terms"]].tolist()
    term_map = {t: i for i, t in enumerate(top_terms)}
    
    label_matrix = np.zeros((len(train_ids), len(top_terms)), dtype=np.float32)
    relevant = aspect_terms[aspect_terms['term'].isin(top_terms)]
    for _, row in tqdm(relevant.iterrows(), total=len(relevant)):
        if row['id'] in id_to_idx:
            label_matrix[id_to_idx[row['id']], term_map[row['term']]] = 1.0
    
    models = [Model_A, Model_B, Model_C]
    model_names = ['Simple', 'Deep', 'Wide+BN']
    
    for idx, (ModelClass, name) in enumerate(zip(models, model_names)):
        print(f"\n  Training {name}...")
        torch.manual_seed(42 + idx)
        
        ds = SimpleData(train_emb, label_matrix)
        loader = DataLoader(ds, batch_size=256, shuffle=True, num_workers=2)
        
        model = ModelClass(1280, len(top_terms)).to(device)
        opt = torch.optim.Adam(model.parameters(), lr=1e-3)
        loss_fn = nn.BCEWithLogitsLoss()
        
        for epoch in range(10):
            model.train()
            total = 0
            for x, y in tqdm(loader, leave=False):
                x, y = x.to(device), y.to(device)
                opt.zero_grad()
                loss = loss_fn(model(x), y)
                loss.backward()
                opt.step()
                total += loss.item()
            if epoch % 3 == 0:
                print(f"    Epoch {epoch+1}: {total/len(loader):.4f}")
        
        torch.save(model.state_dict(), f"model_diverse_{aspect_char}_{idx}.pth")
    
    return top_terms

# Train
results = {}
for char, name in [('F', 'Function'), ('P', 'Process'), ('C', 'Component')]:
    results[char] = train_diverse_ensemble(char, name)

# PREDICT
print("\n=== DIVERSE ENSEMBLE PREDICTION ===")
with open("submission_diverse.tsv", "w") as f:
    for char in ['F', 'P', 'C']:
        print(f"\nEnsembling {char}...")
        terms = results[char]
        models = [Model_A, Model_B, Model_C]
        
        all_preds = []
        for idx, ModelClass in enumerate(models):
            model = ModelClass(1280, len(terms)).to(device)
            model.load_state_dict(torch.load(f"model_diverse_{char}_{idx}.pth"))
            model.eval()
            
            loader = DataLoader(torch.from_numpy(test_emb), batch_size=1024)
            preds = []
            with torch.no_grad():
                for x in tqdm(loader, leave=False):
                    preds.append(torch.sigmoid(model(x.to(device))).cpu().numpy())
            
            all_preds.append(np.vstack(preds))
            del model
        
        # Average
        final_preds = np.mean(all_preds, axis=0)
        
        for i, pid in enumerate(tqdm(test_ids)):
            top_idx = np.argpartition(final_preds[i], -70)[-70:]
            for idx in top_idx:
                if final_preds[i, idx] > 0.01:
                    f.write(f"{pid}\t{terms[idx]}\t{final_preds[i, idx]:.3f}\n")
        
        del all_preds
        gc.collect()

print("\n✅ Done!")


In [ ]:
!pip install obonet -q
import obonet
graph = obonet.read_obo(CONFIG["paths"]["go_obo"])
parent_map = {n: list(graph.successors(n)) for n in graph.nodes()}

sub = {}
with open("submission_diverse.tsv") as f:
    for line in f:
        p, t, s = line.strip().split("\t")
        if p not in sub: sub[p] = {}
        sub[p][t] = float(s)

with open("submissionSimple+Deep+Wide.tsv", "w") as f:
    for pid, preds in tqdm(sub.items()):
        final = preds.copy()
        q = list(preds.keys())
        while q:
            term = q.pop(0)
            for par in parent_map.get(term, []):
                if final.get(par, 0) < final[term]:
                    final[par] = final[term]
                    if par not in preds: q.append(par)
        for t, s in sorted(final.items(), key=lambda x: -x[1])[:70]:
            if s > 0.001:
                f.write(f"{pid}\t{t}\t{s:.3f}\n")


In [ ]:
# === 5-MODEL ENSEMBLE (Conservative) ===
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import numpy as np
import pandas as pd
import gc

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

CONFIG = {
    "n_terms": 1500,
    "n_models": 5,  # INCREASED from 3
    "paths": {
        "train_emb": "/kaggle/input/esm-dataset/train_embeds.npy",
        "train_ids": "/kaggle/input/esm-dataset/train_ids.npy",
        "test_emb": "/kaggle/input/esm-dataset/test_embeds.npy",
        "test_ids": "/kaggle/input/esm-dataset/test_ids.npy",
        "train_terms": "/kaggle/input/cafa-6-protein-function-prediction/Train/train_terms.tsv",
        "go_obo": "/kaggle/input/cafa-6-protein-function-prediction/Train/go-basic.obo"
    }
}

# LOAD DATA
print("Loading...")
train_emb = np.load(CONFIG["paths"]["train_emb"]).astype(np.float32)
train_ids = np.load(CONFIG["paths"]["train_ids"])
test_emb = np.load(CONFIG["paths"]["test_emb"]).astype(np.float32)
test_ids = np.load(CONFIG["paths"]["test_ids"])

mean = train_emb.mean(axis=0)
std = train_emb.std(axis=0) + 1e-6
train_emb = (train_emb - mean) / std
test_emb = (test_emb - mean) / std

terms_df = pd.read_csv(CONFIG["paths"]["train_terms"], sep="\t", header=None, names=["id", "term", "aspect"])
id_to_idx = {pid: i for i, pid in enumerate(train_ids)}

# SIMPLE MODEL (What got you 0.209)
class SimpleModel(nn.Module):
    def __init__(self, n_feat, n_class):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_feat, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, n_class)
        )
    def forward(self, x): return self.net(x)

class SimpleData(Dataset):
    def __init__(self, X, y=None):
        self.X = torch.from_numpy(X)
        self.y = torch.from_numpy(y) if y is not None else None
    def __len__(self): return len(self.X)
    def __getitem__(self, i): return (self.X[i], self.y[i]) if self.y is not None else self.X[i]

# TRAIN
def train_ensemble(aspect_char, aspect_name):
    print(f"\n>>> {aspect_name} - Training {CONFIG['n_models']} models...")
    
    aspect_terms = terms_df[terms_df['aspect'] == aspect_char]
    top_terms = aspect_terms['term'].value_counts().index[:CONFIG["n_terms"]].tolist()
    term_map = {t: i for i, t in enumerate(top_terms)}
    
    label_matrix = np.zeros((len(train_ids), len(top_terms)), dtype=np.float32)
    relevant = aspect_terms[aspect_terms['term'].isin(top_terms)]
    for _, row in tqdm(relevant.iterrows(), total=len(relevant), desc="Labels"):
        if row['id'] in id_to_idx:
            label_matrix[id_to_idx[row['id']], term_map[row['term']]] = 1.0
    
    for model_idx in range(CONFIG["n_models"]):
        print(f"\n  Model {model_idx+1}/{CONFIG['n_models']}...")
        torch.manual_seed(42 + model_idx * 10)
        np.random.seed(42 + model_idx * 10)
        
        ds = SimpleData(train_emb, label_matrix)
        loader = DataLoader(ds, batch_size=256, shuffle=True, num_workers=2)
        
        model = SimpleModel(1280, len(top_terms)).to(device)
        opt = torch.optim.Adam(model.parameters(), lr=1e-3)
        loss_fn = nn.BCEWithLogitsLoss()
        
        for epoch in range(10):
            model.train()
            total = 0
            for x, y in tqdm(loader, leave=False):
                x, y = x.to(device), y.to(device)
                opt.zero_grad()
                loss = loss_fn(model(x), y)
                loss.backward()
                opt.step()
                total += loss.item()
            if epoch % 3 == 0:
                print(f"    Epoch {epoch+1}: {total/len(loader):.4f}")
        
        torch.save(model.state_dict(), f"model_5ens_{aspect_char}_{model_idx}.pth")
    
    return top_terms

# Train
results = {}
for char, name in [('F', 'Function'), ('P', 'Process'), ('C', 'Component')]:
    results[char] = train_ensemble(char, name)

# PREDICT
print("\n=== 5-MODEL ENSEMBLE PREDICTION ===")
with open("submission_5ens.tsv", "w") as f:
    for char in ['F', 'P', 'C']:
        print(f"\nEnsembling {char}...")
        terms = results[char]
        
        all_preds = []
        for model_idx in range(CONFIG["n_models"]):
            model = SimpleModel(1280, len(terms)).to(device)
            model.load_state_dict(torch.load(f"model_5ens_{char}_{model_idx}.pth"))
            model.eval()
            
            loader = DataLoader(torch.from_numpy(test_emb), batch_size=1024)
            preds = []
            with torch.no_grad():
                for x in tqdm(loader, leave=False):
                    preds.append(torch.sigmoid(model(x.to(device))).cpu().numpy())
            
            all_preds.append(np.vstack(preds))
            del model
        
        # Average all 5
        final_preds = np.mean(all_preds, axis=0)
        
        for i, pid in enumerate(tqdm(test_ids)):
            top_idx = np.argpartition(final_preds[i], -70)[-70:]
            for idx in top_idx:
                if final_preds[i, idx] > 0.01:
                    f.write(f"{pid}\t{terms[idx]}\t{final_preds[i, idx]:.3f}\n")
        
        del all_preds
        gc.collect()

print("\n✅ Done!")


In [ ]:
!pip install obonet -q
import obonet
graph = obonet.read_obo(CONFIG["paths"]["go_obo"])
parent_map = {n: list(graph.successors(n)) for n in graph.nodes()}

sub = {}
with open("submission_5ens.tsv") as f:
    for line in f:
        p, t, s = line.strip().split("\t")
        if p not in sub: sub[p] = {}
        sub[p][t] = float(s)

with open("submissionenvdd.tsv", "w") as f:
    for pid, preds in tqdm(sub.items()):
        final = preds.copy()
        q = list(preds.keys())
        while q:
            term = q.pop(0)
            for par in parent_map.get(term, []):
                if final.get(par, 0) < final[term]:
                    final[par] = final[term]
                    if par not in preds: q.append(par)
        for t, s in sorted(final.items(), key=lambda x: -x[1])[:70]:
            if s > 0.001:
                f.write(f"{pid}\t{t}\t{s:.3f}\n")


In [ ]:
# === BASELINE: 3-MODEL ENSEMBLE ===
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import numpy as np
import pandas as pd
import gc

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

CONFIG = {
    "n_terms": 1500,
    "n_models": 3,
    "paths": {
        "train_emb": "/kaggle/input/esm-dataset/train_embeds.npy",
        "train_ids": "/kaggle/input/esm-dataset/train_ids.npy",
        "test_emb": "/kaggle/input/esm-dataset/test_embeds.npy",
        "test_ids": "/kaggle/input/esm-dataset/test_ids.npy",
        "train_terms": "/kaggle/input/cafa-6-protein-function-prediction/Train/train_terms.tsv",
        "go_obo": "/kaggle/input/cafa-6-protein-function-prediction/Train/go-basic.obo"
    }
}

# LOAD DATA
print("Loading...")
train_emb = np.load(CONFIG["paths"]["train_emb"]).astype(np.float32)
train_ids = np.load(CONFIG["paths"]["train_ids"])
test_emb = np.load(CONFIG["paths"]["test_emb"]).astype(np.float32)
test_ids = np.load(CONFIG["paths"]["test_ids"])

mean = train_emb.mean(axis=0)
std = train_emb.std(axis=0) + 1e-6
train_emb = (train_emb - mean) / std
test_emb = (test_emb - mean) / std

terms_df = pd.read_csv(CONFIG["paths"]["train_terms"], sep="\t", header=None, names=["id", "term", "aspect"])
id_to_idx = {pid: i for i, pid in enumerate(train_ids)}

# MODEL
class SimpleModel(nn.Module):
    def __init__(self, n_feat, n_class):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_feat, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, n_class)
        )
    def forward(self, x): return self.net(x)

class SimpleData(Dataset):
    def __init__(self, X, y=None):
        self.X = torch.from_numpy(X)
        self.y = torch.from_numpy(y) if y is not None else None
    def __len__(self): return len(self.X)
    def __getitem__(self, i): return (self.X[i], self.y[i]) if self.y is not None else self.X[i]

# TRAIN
def train_ensemble(aspect_char, aspect_name):
    print(f"\n>>> {aspect_name}...")
    
    aspect_terms = terms_df[terms_df['aspect'] == aspect_char]
    top_terms = aspect_terms['term'].value_counts().index[:CONFIG["n_terms"]].tolist()
    term_map = {t: i for i, t in enumerate(top_terms)}
    
    label_matrix = np.zeros((len(train_ids), len(top_terms)), dtype=np.float32)
    relevant = aspect_terms[aspect_terms['term'].isin(top_terms)]
    for _, row in tqdm(relevant.iterrows(), total=len(relevant)):
        if row['id'] in id_to_idx:
            label_matrix[id_to_idx[row['id']], term_map[row['term']]] = 1.0
    
    for model_idx in range(CONFIG["n_models"]):
        print(f"  Training model {model_idx+1}/3...")
        torch.manual_seed(42 + model_idx)
        
        ds = SimpleData(train_emb, label_matrix)
        loader = DataLoader(ds, batch_size=256, shuffle=True, num_workers=2)
        
        model = SimpleModel(1280, len(top_terms)).to(device)
        opt = torch.optim.Adam(model.parameters(), lr=1e-3)
        loss_fn = nn.BCEWithLogitsLoss()
        
        for epoch in range(10):
            model.train()
            total = 0
            for x, y in tqdm(loader, leave=False):
                x, y = x.to(device), y.to(device)
                opt.zero_grad()
                loss = loss_fn(model(x), y)
                loss.backward()
                opt.step()
                total += loss.item()
        
        torch.save(model.state_dict(), f"model_{aspect_char}_{model_idx}.pth")
    
    return top_terms

# Train all aspects
results = {}
for char, name in [('F', 'Function'), ('P', 'Process'), ('C', 'Component')]:
    results[char] = train_ensemble(char, name)

# GENERATE ENSEMBLE PREDICTIONS (Save to dictionary)
print("\n=== Generating Base Predictions ===")
predictions = {}  # {aspect: {protein_id: {term: score}}}

for char in ['F', 'P', 'C']:
    print(f"\nEnsembling {char}...")
    terms = results[char]
    
    all_preds = []
    for model_idx in range(CONFIG["n_models"]):
        model = SimpleModel(1280, len(terms)).to(device)
        model.load_state_dict(torch.load(f"model_{char}_{model_idx}.pth"))
        model.eval()
        
        loader = DataLoader(torch.from_numpy(test_emb), batch_size=1024)
        preds = []
        with torch.no_grad():
            for x in tqdm(loader, leave=False):
                preds.append(torch.sigmoid(model(x.to(device))).cpu().numpy())
        
        all_preds.append(np.vstack(preds))
        del model
    
    # Average
    final_preds = np.mean(all_preds, axis=0)
    
    # Store in dictionary
    predictions[char] = {}
    for i, pid in enumerate(test_ids):
        predictions[char][pid] = {terms[j]: final_preds[i, j] for j in range(len(terms))}
    
    del all_preds
    gc.collect()

print("\n✅ Base predictions ready!")


In [ ]:
# === SYSTEMATIC POST-PROCESSING OPTIMIZATION ===
!pip install obonet networkx -q

import obonet
from itertools import product

# Load GO graph
print("Loading GO graph...")
graph = obonet.read_obo(CONFIG["paths"]["go_obo"])
parent_map = {n: list(graph.successors(n)) for n in graph.nodes()}

# Test configurations
thresholds = [0.005, 0.008, 0.01, 0.015]
topk_before = [60, 70, 80]
topk_after = [70, 80, 100]

print(f"\n=== Testing {len(thresholds) * len(topk_before) * len(topk_after)} variants ===\n")

variant_count = 0
for thresh, k_before, k_after in product(thresholds, topk_before, topk_after):
    variant_count += 1
    filename = f"sub_t{int(thresh*1000)}_kb{k_before}_ka{k_after}.tsv"
    
    print(f"[{variant_count}] thresh={thresh}, topk_pre={k_before}, topk_post={k_after}")
    
    with open(filename, "w") as f:
        for char in ['F', 'P', 'C']:
            terms = results[char]
            
            for pid in tqdm(test_ids, leave=False):
                pred_dict = predictions[char][pid]
                
                # Filter by threshold and select top-k BEFORE propagation
                filtered = {t: s for t, s in pred_dict.items() if s > thresh}
                if len(filtered) > k_before:
                    sorted_items = sorted(filtered.items(), key=lambda x: -x[1])
                    filtered = dict(sorted_items[:k_before])
                
                # Propagate
                final_scores = filtered.copy()
                queue = list(filtered.keys())
                visited = set(queue)
                
                while queue:
                    term = queue.pop(0)
                    score = final_scores.get(term, 0.0)
                    if term in parent_map:
                        for parent in parent_map[term]:
                            old = final_scores.get(parent, 0.0)
                            if score > old:
                                final_scores[parent] = score
                                if parent not in visited:
                                    queue.append(parent)
                                    visited.add(parent)
                
                # Select top-k AFTER propagation
                sorted_final = sorted(final_scores.items(), key=lambda x: -x[1])[:k_after]
                
                for term, score in sorted_final:
                    if score > 0.001:
                        f.write(f"{pid}\t{term}\t{score:.3f}\n")
    
    print(f"  ✓ Saved to {filename}")

print(f"\n✅ Generated {variant_count} submission variants!")
print("\nNow submit each file and track which performs best.")


In [1]:
# ============================================
# PSEUDO-LABELING: 0.209 → 0.25+ Expected
# Uses your existing embeddings + models
# ============================================

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import numpy as np
import pandas as pd
import gc

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

# Paths
PATHS = {
    "train_emb": "/kaggle/input/esm-dataset/train_embeds.npy",
    "train_ids": "/kaggle/input/esm-dataset/train_ids.npy",
    "test_emb": "/kaggle/input/esm-dataset/test_embeds.npy",
    "test_ids": "/kaggle/input/esm-dataset/test_ids.npy",
    "train_terms": "/kaggle/input/cafa-6-protein-function-prediction/Train/train_terms.tsv",
    "go_obo": "/kaggle/input/cafa-6-protein-function-prediction/Train/go-basic.obo"
}

# Load data
print("Loading embeddings...")
train_emb = np.load(PATHS["train_emb"]).astype(np.float32)
train_ids = np.load(PATHS["train_ids"])
test_emb = np.load(PATHS["test_emb"]).astype(np.float32)
test_ids = np.load(PATHS["test_ids"])

# Normalize
mean = train_emb.mean(axis=0)
std = train_emb.std(axis=0) + 1e-6
train_emb = (train_emb - mean) / std
test_emb = (test_emb - mean) / std

print(f"✓ Train: {train_emb.shape}")
print(f"✓ Test: {test_emb.shape}")

# Load GO terms
terms_df = pd.read_csv(PATHS["train_terms"], sep="\t", header=None, names=["id", "term", "aspect"])
id_to_idx = {pid: i for i, pid in enumerate(train_ids)}

print(f"✓ GO annotations: {len(terms_df):,} rows")


Device: cuda
Loading embeddings...
✓ Train: (82404, 1280)
✓ Test: (224309, 1280)
✓ GO annotations: 537,028 rows


In [2]:
# ============================================
# MODEL ARCHITECTURE
# ============================================

class SimpleModel(nn.Module):
    def __init__(self, n_feat, n_class):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_feat, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, n_class)
        )
    
    def forward(self, x):
        return self.net(x)

class SimpleDataset(Dataset):
    def __init__(self, X, y=None):
        self.X = torch.from_numpy(X)
        self.y = torch.from_numpy(y) if y is not None else None
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        if self.y is not None:
            return self.X[idx], self.y[idx]
        return self.X[idx]

print("✓ Model architecture ready")


✓ Model architecture ready


In [3]:
# ============================================
# TRAIN BASE MODELS (3-Model Ensemble)
# This recreates your 0.209 baseline
# ============================================

def train_aspect(aspect_char, n_terms=1500, n_models=3):
    """Train ensemble for one GO aspect"""
    
    print(f"\n{'='*50}")
    print(f"TRAINING: Aspect {aspect_char}")
    print(f"{'='*50}")
    
    # Get top terms
    aspect_terms = terms_df[terms_df['aspect'] == aspect_char]
    top_terms = aspect_terms['term'].value_counts().index[:n_terms].tolist()
    term_map = {t: i for i, t in enumerate(top_terms)}
    
    # Build label matrix
    label_matrix = np.zeros((len(train_ids), len(top_terms)), dtype=np.float32)
    relevant = aspect_terms[aspect_terms['term'].isin(top_terms)]
    
    for _, row in tqdm(relevant.iterrows(), desc="Building labels", total=len(relevant)):
        if row['id'] in id_to_idx:
            label_matrix[id_to_idx[row['id']], term_map[row['term']]] = 1.0
    
    print(f"✓ Label matrix: {label_matrix.shape}, sparsity: {(label_matrix > 0).mean():.4f}")
    
    # Train ensemble
    for model_idx in range(n_models):
        print(f"\n  Model {model_idx+1}/{n_models}:")
        torch.manual_seed(42 + model_idx)
        
        dataset = SimpleDataset(train_emb, label_matrix)
        loader = DataLoader(dataset, batch_size=256, shuffle=True, num_workers=2)
        
        model = SimpleModel(1280, len(top_terms)).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
        criterion = nn.BCEWithLogitsLoss()
        
        for epoch in range(10):
            model.train()
            total_loss = 0
            
            for X_batch, y_batch in tqdm(loader, desc=f"    Epoch {epoch+1}/10", leave=False):
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                
                optimizer.zero_grad()
                loss = criterion(model(X_batch), y_batch)
                loss.backward()
                optimizer.step()
                
                total_loss += loss.item()
            
            if (epoch + 1) % 3 == 0:
                print(f"    Epoch {epoch+1}: Loss = {total_loss/len(loader):.4f}")
        
        # Save model
        torch.save(model.state_dict(), f"base_model_{aspect_char}_{model_idx}.pth")
        del model
        gc.collect()
        torch.cuda.empty_cache()
    
    return top_terms

# Train all aspects
print("STAGE 1: Training Base Models")
print("="*60)

results = {}
for char in ['F', 'P', 'C']:
    results[char] = train_aspect(char)

print("\n✅ Base models trained and saved!")


STAGE 1: Training Base Models

TRAINING: Aspect F


Building labels: 100%|██████████| 112061/112061 [00:04<00:00, 24588.62it/s]


✓ Label matrix: (82404, 1500), sparsity: 0.0009

  Model 1/3:


    Epoch 3: Loss = 0.0040


    Epoch 6: Loss = 0.0033


...


  Model 3/3:


    Epoch 3: Loss = 0.0053


    Epoch 6: Loss = 0.0047


    Epoch 9: Loss = 0.0044



✅ Base models trained and saved!


In [4]:
# ============================================
# PSEUDO-LABELING: Generate high-confidence predictions
# ============================================

print("\nSTAGE 2: Generating Pseudo-Labels")
print("="*60)

CONFIDENCE_THRESHOLD = 0.75  # Only use very confident predictions

pseudo_labels = {char: {} for char in ['F', 'P', 'C']}  # {aspect: {protein_id: {term: score}}}

for char in ['F', 'P', 'C']:
    print(f"\nProcessing aspect {char}...")
    terms = results[char]
    
    # Load models and ensemble predict
    all_preds = []
    
    for model_idx in range(3):
        model = SimpleModel(1280, len(terms)).to(device)
        model.load_state_dict(torch.load(f"base_model_{char}_{model_idx}.pth"))
        model.eval()
        
        preds = []
        test_loader = DataLoader(torch.from_numpy(test_emb), batch_size=1024, num_workers=2)
        
        with torch.no_grad():
            for X_batch in tqdm(test_loader, desc=f"  Model {model_idx+1}", leave=False):
                preds.append(torch.sigmoid(model(X_batch.to(device))).cpu().numpy())
        
        all_preds.append(np.vstack(preds))
        del model
    
    # Average ensemble
    final_preds = np.mean(all_preds, axis=0)
    del all_preds
    gc.collect()
    
    # Extract high-confidence predictions
    high_conf_count = 0
    for i, protein_id in enumerate(test_ids):
        high_conf_indices = np.where(final_preds[i] > CONFIDENCE_THRESHOLD)[0]
        
        if len(high_conf_indices) > 0:
            pseudo_labels[char][protein_id] = {}
            for idx in high_conf_indices:
                pseudo_labels[char][protein_id][terms[idx]] = final_preds[i, idx]
            high_conf_count += len(high_conf_indices)
    
    print(f"  ✓ Generated {high_conf_count:,} pseudo-labels for aspect {char}")

total_pseudo = sum(len(v) for char in pseudo_labels for v in pseudo_labels[char].values())
print(f"\n✅ Total pseudo-labels: {total_pseudo:,}")



STAGE 2: Generating Pseudo-Labels

Processing aspect F...


  ✓ Generated 5,854 pseudo-labels for aspect F

Processing aspect P...


  ✓ Generated 1,406 pseudo-labels for aspect P

Processing aspect C...


  ✓ Generated 13,073 pseudo-labels for aspect C

✅ Total pseudo-labels: 20,333


In [5]:
# ============================================
# AUGMENT TRAINING DATA with Pseudo-Labels
# ============================================

print("\nSTAGE 3: Creating Augmented Training Set")
print("="*60)

# Create augmented embeddings and labels
# We'll add a SUBSET of test proteins with high-confidence predictions

PSEUDO_PROTEINS_PER_ASPECT = 5000  # Limit to avoid overfitting

augmented_data = {}

for char in ['F', 'P', 'C']:
    print(f"\nAspect {char}:")
    terms = results[char]
    term_map = {t: i for i, t in enumerate(terms)}
    
    # Select top pseudo-labeled proteins (most confident)
    protein_confidences = {}
    for pid, term_scores in pseudo_labels[char].items():
        protein_confidences[pid] = np.mean(list(term_scores.values()))
    
    # Sort by confidence and take top N
    sorted_proteins = sorted(protein_confidences.items(), key=lambda x: -x[1])
    selected_proteins = [pid for pid, conf in sorted_proteins[:PSEUDO_PROTEINS_PER_ASPECT]]
    
    print(f"  Selected {len(selected_proteins):,} pseudo-labeled proteins")
    
    # Get their embeddings
    test_id_to_idx = {pid: i for i, pid in enumerate(test_ids)}
    pseudo_indices = [test_id_to_idx[pid] for pid in selected_proteins if pid in test_id_to_idx]
    pseudo_emb = test_emb[pseudo_indices]
    
    # Build pseudo labels
    pseudo_label_matrix = np.zeros((len(pseudo_indices), len(terms)), dtype=np.float32)
    for i, pid in enumerate(selected_proteins[:len(pseudo_indices)]):
        for term, score in pseudo_labels[char][pid].items():
            if term in term_map:
                pseudo_label_matrix[i, term_map[term]] = score
    
    # Combine with original training data
    original_labels = np.zeros((len(train_ids), len(terms)), dtype=np.float32)
    aspect_terms = terms_df[terms_df['aspect'] == char]
    relevant = aspect_terms[aspect_terms['term'].isin(terms)]
    
    for _, row in relevant.iterrows():
        if row['id'] in id_to_idx and row['term'] in term_map:
            original_labels[id_to_idx[row['id']], term_map[row['term']]] = 1.0
    
    # Stack
    augmented_emb = np.vstack([train_emb, pseudo_emb])
    augmented_labels = np.vstack([original_labels, pseudo_label_matrix])
    
    augmented_data[char] = {
        'embeddings': augmented_emb,
        'labels': augmented_labels,
        'terms': terms
    }
    
    print(f"  ✓ Augmented: {augmented_emb.shape[0]:,} total proteins")
    print(f"  ✓ Added {len(pseudo_indices):,} pseudo-labeled samples")

print("\n✅ Augmented datasets ready!")



STAGE 3: Creating Augmented Training Set

Aspect F:
  Selected 5,000 pseudo-labeled proteins
  ✓ Augmented: 87,404 total proteins
  ✓ Added 5,000 pseudo-labeled samples

Aspect P:
  Selected 1,314 pseudo-labeled proteins
  ✓ Augmented: 83,718 total proteins
  ✓ Added 1,314 pseudo-labeled samples

Aspect C:
  Selected 5,000 pseudo-labeled proteins
  ✓ Augmented: 87,404 total proteins
  ✓ Added 5,000 pseudo-labeled samples

✅ Augmented datasets ready!


In [6]:
# ============================================
# RETRAIN MODELS on Augmented Data
# ============================================

print("\nSTAGE 4: Retraining on Augmented Data")
print("="*60)

for char in ['F', 'P', 'C']:
    print(f"\n{'='*50}")
    print(f"RETRAINING: Aspect {char}")
    print(f"{'='*50}")
    
    data = augmented_data[char]
    X_aug = data['embeddings']
    y_aug = data['labels']
    terms = data['terms']
    
    # Train new ensemble
    for model_idx in range(3):
        print(f"\n  Model {model_idx+1}/3:")
        torch.manual_seed(100 + model_idx)  # Different seed than base
        
        dataset = SimpleDataset(X_aug, y_aug)
        loader = DataLoader(dataset, batch_size=256, shuffle=True, num_workers=2)
        
        model = SimpleModel(1280, len(terms)).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=8e-4)  # Slightly lower LR
        criterion = nn.BCEWithLogitsLoss()
        
        for epoch in range(12):  # More epochs
            model.train()
            total_loss = 0
            
            for X_batch, y_batch in tqdm(loader, desc=f"    Epoch {epoch+1}/12", leave=False):
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                
                optimizer.zero_grad()
                loss = criterion(model(X_batch), y_batch)
                loss.backward()
                optimizer.step()
                
                total_loss += loss.item()
            
            if (epoch + 1) % 3 == 0:
                print(f"    Epoch {epoch+1}: Loss = {total_loss/len(loader):.4f}")
        
        # Save
        torch.save(model.state_dict(), f"pseudo_model_{char}_{model_idx}.pth")
        del model
        gc.collect()
        torch.cuda.empty_cache()

print("\n✅ Pseudo-labeled models trained!")



STAGE 4: Retraining on Augmented Data

RETRAINING: Aspect F

  Model 1/3:


    Epoch 3: Loss = 0.0039


    Epoch 6: Loss = 0.0032


    Epoch 9: Loss = 0.0029


    Epoch 12: Loss = 0.0027

  Model 2/3:


...

    Epoch 12: Loss = 0.0040

  Model 3/3:


    Epoch 3: Loss = 0.0052


    Epoch 6: Loss = 0.0046


    Epoch 9: Loss = 0.0042


    Epoch 12: Loss = 0.0040

✅ Pseudo-labeled models trained!


In [7]:
# ============================================
# GENERATE SUBMISSION with Pseudo-Labeled Models
# ============================================

!pip install obonet -q
import obonet

print("\nSTAGE 5: Generating Final Submission")
print("="*60)

# Load GO graph for propagation
graph = obonet.read_obo(PATHS["go_obo"])
parent_map = {n: list(graph.successors(n)) for n in graph.nodes()}

# Prediction parameters
THRESHOLD = 0.01
TOPK_BEFORE_PROP = 80
TOPK_AFTER_PROP = 80

with open("submission_pseudo.tsv", "w") as f:
    for char in ['F', 'P', 'C']:
        print(f"\nProcessing aspect {char}...")
        terms = results[char]
        
        # Ensemble predict with NEW models
        all_preds = []
        for model_idx in range(3):
            model = SimpleModel(1280, len(terms)).to(device)
            model.load_state_dict(torch.load(f"pseudo_model_{char}_{model_idx}.pth"))
            model.eval()
            
            preds = []
            test_loader = DataLoader(torch.from_numpy(test_emb), batch_size=1024)
            
            with torch.no_grad():
                for X_batch in tqdm(test_loader, desc=f"  Model {model_idx+1}", leave=False):
                    preds.append(torch.sigmoid(model(X_batch.to(device))).cpu().numpy())
            
            all_preds.append(np.vstack(preds))
            del model
        
        final_preds = np.mean(all_preds, axis=0)
        del all_preds
        gc.collect()
        
        # Process each protein
        for i, protein_id in enumerate(tqdm(test_ids, desc="  Writing", leave=False)):
            # Filter and top-k
            scores = {terms[j]: final_preds[i, j] for j in range(len(terms)) if final_preds[i, j] > THRESHOLD}
            if len(scores) > TOPK_BEFORE_PROP:
                scores = dict(sorted(scores.items(), key=lambda x: -x[1])[:TOPK_BEFORE_PROP])
            
            # Propagate
            final_scores = scores.copy()
            queue = list(scores.keys())
            visited = set(queue)
            
            while queue:
                term = queue.pop(0)
                score = final_scores.get(term, 0)
                for parent in parent_map.get(term, []):
                    if final_scores.get(parent, 0) < score:
                        final_scores[parent] = score
                        if parent not in visited:
                            queue.append(parent)
                            visited.add(parent)
            
            # Write top results
            for term, score in sorted(final_scores.items(), key=lambda x: -x[1])[:TOPK_AFTER_PROP]:
                if score > 0.001:
                    f.write(f"{protein_id}\t{term}\t{score:.3f}\n")

print("\n✅ Submission file created: submission_pseudo.tsv")
print("Expected improvement: 0.209 → 0.24-0.26")



STAGE 5: Generating Final Submission

Processing aspect F...



Processing aspect P...



Processing aspect C...



✅ Submission file created: submission_pseudo.tsv
Expected improvement: 0.209 → 0.24-0.26
